# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{38138448510: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #objects:1>

In [14]:
x.id_at_location

38138448510

In [15]:
x.id

81579208690

In [16]:
x.owner

<VirtualWorker id:me #objects:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:81579208690 -> bob:38138448510]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [21]:
# try this project here!

In [2]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)

In [3]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [8]:
bob._objects, alice._objects

({40488058247: tensor([1, 2, 3, 4, 5])},
 {60149285284: tensor([1, 2, 3, 4, 5])})

In [6]:
x = th.tensor([1,2,3,4,5])

In [26]:
x_ = x.send(bob)
y_ = y.send(alice)
x_, y_

((Wrapper)>[PointerTensor | me:97574164572 -> bob:70875861257],
 (Wrapper)>[PointerTensor | me:28997453084 -> alice:94054505782])

In [27]:
bob._objects, alice._objects

({70875861257: tensor([1, 2, 3, 4, 5])},
 {94054505782: tensor([ 2,  4,  6,  8, 10])})

In [28]:
x = x_.get()
y = y_.get()
x, y

(tensor([1, 2, 3, 4, 5]), tensor([ 2,  4,  6,  8, 10]))

In [29]:
bob._objects, alice._objects

({}, {})

In [30]:
x = th.tensor([1,2,3,4,5])

In [31]:
x_ptr = x.send(bob, alice)
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:38847167884 -> bob:63250315560]
	-> (Wrapper)>[PointerTensor | me:32670601565 -> alice:28190920743]

In [32]:
x, y = x_ptr.get()
x, y

(tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5]))

In [33]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [34]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [35]:
x

(Wrapper)>[PointerTensor | me:80451470605 -> bob:15506970573]

In [36]:
y

(Wrapper)>[PointerTensor | me:88593981939 -> bob:35580790892]

In [37]:
z = x + y

In [38]:
z

(Wrapper)>[PointerTensor | me:42295993078 -> bob:67817955946]

In [39]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [40]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:8651251039 -> bob:27955348978]

In [41]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [42]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [43]:
z = (x + y).sum()

In [44]:
z.backward()

(Wrapper)>[PointerTensor | me:4355233222 -> bob:54768920038]

In [45]:
x = x.get()

In [46]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [47]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [48]:
# try this project here!

In [49]:
inputs = th.tensor([[1., 1], [1, 0], [0, 1], [0, 0]], requires_grad=True).send(bob)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [50]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [51]:
for i in range(20):
    preds = inputs.mm(weights)
    loss = ((preds - target)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)
tensor(0.0058)
tensor(0.0037)
tensor(0.0024)
tensor(0.0015)
tensor(0.0010)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0001)


# Lesson: Garbage Collection and Common Errors


In [52]:
bob = bob.clear_objects()

In [53]:
bob._objects

{}

In [54]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
bob._objects

{}

In [56]:
del x

In [57]:
bob._objects

{}

In [58]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [59]:
bob._objects

{}

In [60]:
x = "asdf"

In [61]:
bob._objects

{}

In [62]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [63]:
x

(Wrapper)>[PointerTensor | me:69247589805 -> bob:23906319812]

In [64]:
bob._objects

{}

In [65]:
x = "asdf"

In [66]:
bob._objects

{}

In [67]:
del x

In [68]:
bob._objects

{}

In [69]:
bob = bob.clear_objects()
bob._objects

{}

In [70]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [71]:
bob._objects

{}

In [72]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [73]:
# z = x + y

In [74]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [75]:
# z = x + y

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [76]:
from torch import nn, optim

In [77]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [78]:
# A Toy Model
model = nn.Linear(2,1)

In [79]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [80]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(14.5178)
tensor(1.7513)
tensor(0.5835)
tensor(0.3563)
tensor(0.2446)
tensor(0.1715)
tensor(0.1213)
tensor(0.0865)
tensor(0.0622)
tensor(0.0450)
tensor(0.0327)
tensor(0.0240)
tensor(0.0177)
tensor(0.0131)
tensor(0.0097)
tensor(0.0073)
tensor(0.0054)
tensor(0.0041)
tensor(0.0031)
tensor(0.0023)


In [81]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [82]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [83]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [84]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [85]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [86]:
train()

tensor(0.5638, requires_grad=True)
tensor(2.4042, requires_grad=True)
tensor(0.8068, requires_grad=True)
tensor(1.3839, requires_grad=True)
tensor(0.4982, requires_grad=True)
tensor(0.8311, requires_grad=True)
tensor(0.2947, requires_grad=True)
tensor(0.5037, requires_grad=True)
tensor(0.1749, requires_grad=True)
tensor(0.3075, requires_grad=True)
tensor(0.1047, requires_grad=True)
tensor(0.1894, requires_grad=True)
tensor(0.0633, requires_grad=True)
tensor(0.1177, requires_grad=True)
tensor(0.0388, requires_grad=True)
tensor(0.0739, requires_grad=True)
tensor(0.0241, requires_grad=True)
tensor(0.0469, requires_grad=True)
tensor(0.0152, requires_grad=True)
tensor(0.0302, requires_grad=True)
tensor(0.0098, requires_grad=True)
tensor(0.0196, requires_grad=True)
tensor(0.0065, requires_grad=True)
tensor(0.0129, requires_grad=True)
tensor(0.0043, requires_grad=True)
tensor(0.0086, requires_grad=True)
tensor(0.0029, requires_grad=True)
tensor(0.0058, requires_grad=True)
tensor(0.0020, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [87]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [88]:
x = th.tensor([1,2,3,4,5]).send(bob)
x

(Wrapper)>[PointerTensor | me:27777595506 -> bob:75863292489]

In [89]:
x = x.send(alice)

In [90]:
bob._objects

{}

In [91]:
alice._objects

{}

In [92]:
# y = x + x

In [93]:
y

(Wrapper)>[PointerTensor | me:56326806452 -> alice:9968076059]

In [94]:
bob._objects

{}

In [95]:
alice._objects

{}

In [96]:
jon = sy.VirtualWorker(hook, id="jon")

In [97]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [98]:
bob._objects

{}

In [99]:
alice._objects

{}

In [100]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:24492485155 -> bob:47018604556]

In [101]:
bob._objects

{}

In [102]:
alice._objects

{}

In [103]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [104]:
bob._objects

{}

In [105]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [106]:
bob._objects

{}

In [107]:
alice._objects

{}

In [108]:
del x

In [109]:
bob._objects

{}

In [110]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [111]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [112]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [113]:
bob._objects

{}

In [114]:
alice._objects

{}

In [115]:
x.move(alice)

KeyError: 'Object "29500002573" not found on worker!!!You just tried to interact with an object ID:29500002573 on <VirtualWorker id:alice #objects:0> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [116]:
bob._objects

{}

In [117]:
alice._objects

{}

In [118]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [119]:
bob._objects

{}

In [120]:
alice._objects

{}

In [121]:
x.remote_get()

KeyError: 'Object "87784206518" not found on worker!!!You just tried to interact with an object ID:87784206518 on <VirtualWorker id:alice #objects:0> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [122]:
bob._objects

{}

In [123]:
alice._objects

{}

In [124]:
x.move(bob)

KeyError: 'Object "2315320305" not found on worker!!!You just tried to interact with an object ID:2315320305 on <VirtualWorker id:bob #objects:0> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [125]:
x

(Wrapper)>[PointerTensor | me:2315320305 -> alice:87784206518]

In [126]:
bob._objects

{}

In [127]:
alice._objects

{}